# Experimento 6
***
- Rede Inception
- Conjunto de Dados: NIH
- Analisando o treinamento para detecção de anormalidades

### Importação dos pacotes

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, accuracy_score
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf

import warnings
warnings.filterwarnings("ignore")

### Importação dos dados
***
- Todo o dataset foi pré-processado anteriormente

In [3]:
# importando os dataframes dos dados de treinamento, validação e teste
train_df = pd.read_csv('/content/drive/MyDrive/train_df.csv', sep = ',', index_col=  0)
validation_df = pd.read_csv('/content/drive/MyDrive/validation_df.csv', sep = ',', index_col=  0)
test_df = pd.read_csv('/content/drive/MyDrive/test_df.csv' , sep = ',', index_col=  0)

In [4]:
# tornando as classes na coluna 'labels' categórica
train_df.loc[train_df.labels == 1, 'labels'] = 'abnormal'
train_df.loc[train_df.labels == 0, 'labels'] = 'normal'

validation_df.loc[validation_df.labels == 1, 'labels'] = 'abnormal'
validation_df.loc[validation_df.labels == 0, 'labels'] = 'normal'

test_df.loc[test_df.labels == 1, 'labels'] = 'abnormal'
test_df.loc[test_df.labels == 0, 'labels'] = 'normal'

In [5]:
# ajustando a quantidade de exemplos nos dados de treinamento relativo a radiográficas normais
train_normal_cases = train_df[train_df.labels == 'normal']
train_abnormal_cases = train_df[train_df.labels == 'abnormal']
train_normal_cases, _ = train_test_split(train_normal_cases, test_size = 0.5, random_state = 42)
train = np.concatenate((train_normal_cases, train_abnormal_cases))
train_df = pd.DataFrame(train, columns = ['Image Index', 'finding_labels', 'labels'])

In [6]:
# visualizando como é a organização do dataframe pré-processado
train_df.head()

,Image Index,finding_labels,labels
0,/content/drive/MyDrive/images-nih4/images/0002...,No Finding,normal
1,/content/drive/MyDrive/images-nih3/images/0001...,No Finding,normal
2,/content/drive/MyDrive/images-nih2/images/0000...,No Finding,normal
3,/content/drive/MyDrive/images-nih2/images/0000...,No Finding,normal
4,/content/drive/MyDrive/images-nih2/images/0001...,No Finding,normal


In [7]:
# visualizando a quantidade de imagens disponíveis por classe
train_df['labels'].value_counts()

normal      27811
abnormal    25452
Name: labels, dtype: int64

In [8]:
# organizando um dicionário para realizar o balanceamento nos dados das classes
class_weights = class_weight.compute_class_weight('balanced', np.unique(train_df['labels']),
                                                  train_df['labels'])
class_weight = {0: class_weights[0], 1: class_weights[1]}

In [9]:
# normalizando as imagens de treinamento e aplicando aumento de dados
image_generator = ImageDataGenerator(rescale = 1./255., rotation_range = 10, zoom_range = 0.2)

# criando o gerador de imagens de treinamento 
train_generator = image_generator.flow_from_dataframe(
                                                      dataframe = train_df,
                                                      x_col = 'Image Index',
                                                      y_col = 'labels',
                                                      batch_size = 256,
                                                      seed = 42,
                                                      shuffle = True,
                                                      class_mode = 'categorical',
                                                      color_mode = 'rgb',
                                                      target_size = (256, 256))

# normalizando as imagens de teste 
test_datagen = ImageDataGenerator(rescale = 1./255.)

# criando o gerador de imagens de validação 
valid_generator = test_datagen.flow_from_dataframe(
                                                    dataframe = validation_df,
                                                    x_col = 'Image Index',
                                                    y_col = 'labels',
                                                    batch_size = 128,
                                                    seed = 42,
                                                    shuffle = True,
                                                    class_mode = 'categorical',
                                                    target_size = (256, 256))

test_generator = test_datagen.flow_from_dataframe(
                                                  dataframe = test_df, 
                                                  x_col = 'Image Index',
                                                  y_col = 'labels',
                                                  batch_size = 128,
                                                  seed = 42,
                                                  shuffle = True,
                                                  class_mode = 'categorical',
                                                  target_size = (256, 256))

Found 53259 validated image filenames belonging to 2 classes.
Found 4268 validated image filenames belonging to 2 classes.
Found 2640 validated image filenames belonging to 2 classes.


In [10]:
# visualizando a ordem numérica das classes nos dados de treinamento
train_generator.class_indices

{'abnormal': 0, 'normal': 1}

### Preparando a rede neural convolucional 

In [18]:
# baixando os pesos treinados da rede inception
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

--2021-04-14 22:49:43--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.119.128, 108.177.126.128, 108.177.127.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.119.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M  70.6MB/s    in 1.2s    

2021-04-14 22:49:44 (70.6 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]



In [19]:
# referenciando o local em que os pesos estão armazenados
local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

# carregando a arquitetura inception pré-treinada
pre_trained_model = InceptionV3(input_shape = (256, 256, 3), include_top = False, weights = None)

# carregando os pesos treinados com outros dados 
pre_trained_model.load_weights(local_weights_file)

# definindo as flags iniciais  
pre_trained_model.trainable = True
set_trainable = False

# para a arquitetura inception, a rede será retreinada a partir da camada 'mixed6'
for layer in pre_trained_model.layers:
    if layer.name == 'mixed6':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

# obtendo a última camada como sendo a nomeada por 'mixed7'
last_layer = pre_trained_model.get_layer('mixed7')
last_output = last_layer.output

In [20]:
# definindo uma camada de achatamento
x = layers.Flatten()(last_output)
# conecatando a rede uma camada com 1024 neurônios e função de ativação relu
x = layers.Dense(units = 1024, activation = tf.nn.relu)(x)     
# conecatando a rede uma camada com 128 neurônios e função de ativação relu
x = layers.Dense(units = 512, activation = tf.nn.relu)(x) 
# aplicando uma camada de dropout com uma taxa de 20% (normalização)
x = layers.Dropout(rate = 0.2)(x)                 
# adicionando uma camada de saída com um neurônio e uma função de ativação sigmoide
x = layers.Dense  (units = 2, activation = tf.nn.softmax)(x)           

# conecatando as camadas definidas acima com a arquitetura inception
model = Model(pre_trained_model.input, x) 

# compilando a rede 
model.compile(optimizer = optimizers.RMSprop(learning_rate = 0.001), loss = 'categorical_crossentropy', 
              metrics = ['acc']) 

In [21]:
# definindo o caminho pelo qual os pesos serão armazenados 
filepath = "/content/drive/MyDrive/weights-nih/inception/transferlearning_weights_bin_v1.0.hdf5"
# callback para salvar o melhor valor dos pesos em relação ao desempenho com os dados de validação 
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc', verbose = 1, save_best_only = True, mode = 'max')

In [ ]:
# definindo um array de callbacks
callbacks = [checkpoint]

In [ ]:
# treinando a rede neural convolucional
history = model.fit_generator(train_generator, steps_per_epoch = 53263 // 256, 
                              validation_data = valid_generator, validation_steps = 4268 // 128,
                              callbacks = callbacks, epochs = 10, class_weight = class_weight,
                              use_multiprocessing = True, workers = 8)

Epoch 1/10
208/208 [==============================] - 3488s 16s/step - loss: 8.1542 - acc: 0.5510 - val_loss: 0.7319 - val_acc: 0.6375

Epoch 00001: val_acc improved from -inf to 0.63755, saving model to /content/drive/MyDrive/weights-nih/inception/transferlearning_weights_bin_v1.0.hdf5
Epoch 2/10
208/208 [==============================] - 1476s 7s/step - loss: 0.6402 - acc: 0.6450 - val_loss: 0.6323 - val_acc: 0.6428

Epoch 00002: val_acc improved from 0.63755 to 0.64276, saving model to /content/drive/MyDrive/weights-nih/inception/transferlearning_weights_bin_v1.0.hdf5
Epoch 3/10
208/208 [==============================] - 1338s 6s/step - loss: 0.6254 - acc: 0.6554 - val_loss: 0.7333 - val_acc: 0.5753

Epoch 00003: val_acc did not improve from 0.64276
Epoch 4/10
208/208 [==============================] - 672s 3s/step - loss: 0.6219 - acc: 0.6596 - val_loss: 0.6698 - val_acc: 0.5665

Epoch 00004: val_acc did not improve from 0.64276
Epoch 5/10
208/208 [==============================] -

In [ ]:
# avaliando a capacidade de predição do modelo com os dados de teste
model.evaluate(test_generator)

21/21 [==============================] - 1831s 91s/step - loss: 0.6030 - acc: 0.6920


[0.6029810309410095, 0.6920454502105713]

In [ ]:
# salvando o modelo treinado
model.save('/content/drive/MyDrive/weights-nih/inception/model_binv1.0')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/weights-nih/inception/model_binv1.0/assets


### Continuação do Treinamento

In [ ]:
# carregando o modelo salvo
model = tf.keras.models.load_model('/content/drive/MyDrive/weights-nih/inception/model_binv1.0')
# carregando os pesos treinados com o modelo
model.load_weights('/content/drive/MyDrive/weights-nih/inception/transferlearning_weights_bin_v1.1.hdf5')

In [ ]:
# definindo o caminho pelo qual os pesos serão armazenados 
filepath = "/content/drive/MyDrive/weights-nih/inception/transferlearning_weights_bin_v1.1.hdf5"
# callback para salvar o melhor valor dos pesos em relação ao desempenho com os dados de validação 
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc', verbose = 1, save_best_only = True, mode = 'max')
# definindo um array de callbacks
callbacks = [checkpoint]

# treinando a rede neural convolucional
history = model.fit_generator(train_generator, steps_per_epoch = 53263 // 256, 
                              validation_data = valid_generator, validation_steps = 4268 // 128,
                              callbacks = callbacks, epochs = 10, class_weight = class_weight,
                              use_multiprocessing = True, workers = 8)

Epoch 1/10
208/208 [==============================] - 4115s 19s/step - loss: 0.5440 - acc: 0.7188 - val_loss: 0.6753 - val_acc: 0.6851

Epoch 00001: val_acc improved from -inf to 0.68513, saving model to /content/drive/MyDrive/weights-nih/inception/transferlearning_weights_bin_v1.1.hdf5
Epoch 2/10
208/208 [==============================] - 1472s 7s/step - loss: 0.5186 - acc: 0.7355 - val_loss: 0.7348 - val_acc: 0.6586

Epoch 00002: val_acc did not improve from 0.68513
Epoch 3/10
208/208 [==============================] - 655s 3s/step - loss: 0.5147 - acc: 0.7366 - val_loss: 0.7393 - val_acc: 0.6402

Epoch 00003: val_acc did not improve from 0.68513
Epoch 4/10
208/208 [==============================] - 637s 3s/step - loss: 0.4995 - acc: 0.7488 - val_loss: 0.6674 - val_acc: 0.6783

Epoch 00004: val_acc did not improve from 0.68513
Epoch 5/10
208/208 [==============================] - 639s 3s/step - loss: 0.4961 - acc: 0.7496 - val_loss: 0.8488 - val_acc: 0.6229

Epoch 00005: val_acc did 

In [ ]:
# definindo o caminho pelo qual os pesos serão armazenados 
filepath = "/content/drive/MyDrive/weights-nih/inception/transferlearning_weights_bin_v1.2.hdf5"
# callback para salvar o melhor valor dos pesos em relação ao desempenho com os dados de validação 
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc', verbose = 1, save_best_only = True, mode = 'max')
# definindo um array de callbacks
callbacks = [checkpoint]

# treinando a rede neural convolucional
history = model.fit_generator(train_generator, steps_per_epoch = 53263 // 256, 
                              validation_data = valid_generator, validation_steps = 4268 // 128,
                              callbacks = callbacks, epochs = 10, class_weight = class_weight,
                              use_multiprocessing = True, workers = 8)

Epoch 1/10
208/208 [==============================] - 639s 3s/step - loss: 0.4556 - acc: 0.7737 - val_loss: 0.8387 - val_acc: 0.6357

Epoch 00001: val_acc improved from -inf to 0.63565, saving model to /content/drive/MyDrive/weights-nih/inception/transferlearning_weights_bin_v1.2.hdf5
Epoch 2/10
208/208 [==============================] - 1196s 6s/step - loss: 0.4487 - acc: 0.7789 - val_loss: 0.7741 - val_acc: 0.6570

Epoch 00002: val_acc improved from 0.63565 to 0.65696, saving model to /content/drive/MyDrive/weights-nih/inception/transferlearning_weights_bin_v1.2.hdf5
Epoch 3/10
208/208 [==============================] - 1266s 6s/step - loss: 0.4426 - acc: 0.7836 - val_loss: 0.7241 - val_acc: 0.6593

Epoch 00003: val_acc improved from 0.65696 to 0.65933, saving model to /content/drive/MyDrive/weights-nih/inception/transferlearning_weights_bin_v1.2.hdf5
Epoch 4/10
208/208 [==============================] - 1315s 6s/step - loss: 0.4355 - acc: 0.7874 - val_loss: 0.8637 - val_acc: 0.6316


In [ ]:
# salvando o modelo treinado
model.save('/content/drive/MyDrive/weights-nih/inception/model_binv1.2')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/weights-nih/inception/model_binv1.2/assets


In [ ]:
# testando a capacidade de predição do modelo com os dados de teste
model.evaluate(test_generator)

21/21 [==============================] - 1845s 92s/step - loss: 0.8507 - acc: 0.6443


[0.850670576095581, 0.644318163394928]

### Continuação da etapa de treinamento

In [22]:
# carregando os pesos previamente treinados
model.load_weights('/content/drive/MyDrive/weights-nih/inception/transferlearning_weights_bin_v1.2.hdf5')

In [23]:
# definindo o caminho pelo qual os pesos serão armazenados 
filepath = "/content/drive/MyDrive/weights-nih/inception/transferlearning_weights_bin_v1.3.hdf5"
# callback para salvar o melhor valor dos pesos em relação ao desempenho com os dados de validação 
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc', verbose = 1, save_best_only = True, mode = 'max')
# definindo um array de callbacks
callbacks = [checkpoint]

# treinando a rede neural convolucional
history = model.fit_generator(train_generator, steps_per_epoch = 53263 // 256, 
                              validation_data = valid_generator, validation_steps = 4268 // 128,
                              callbacks = callbacks, epochs = 10, class_weight = class_weight,
                              use_multiprocessing = True, workers = 8)

Epoch 1/10
208/208 [==============================] - 5856s 28s/step - loss: 0.4353 - acc: 0.7957 - val_loss: 0.8334 - val_acc: 0.6366

Epoch 00001: val_acc improved from -inf to 0.63660, saving model to /content/drive/MyDrive/weights-nih/inception/transferlearning_weights_bin_v1.3.hdf5
Epoch 2/10
208/208 [==============================] - 2721s 13s/step - loss: 0.3980 - acc: 0.8113 - val_loss: 0.8280 - val_acc: 0.6480

Epoch 00002: val_acc improved from 0.63660 to 0.64796, saving model to /content/drive/MyDrive/weights-nih/inception/transferlearning_weights_bin_v1.3.hdf5
Epoch 3/10
208/208 [==============================] - 2014s 10s/step - loss: 0.3886 - acc: 0.8174 - val_loss: 0.9707 - val_acc: 0.6354

Epoch 00003: val_acc did not improve from 0.64796
Epoch 4/10
208/208 [==============================] - 741s 3s/step - loss: 0.3840 - acc: 0.8200 - val_loss: 0.9529 - val_acc: 0.6070

Epoch 00004: val_acc did not improve from 0.64796
Epoch 5/10
208/208 [==============================]

In [24]:
# salvando o modelo treinado
model.save('/content/drive/MyDrive/weights-nih/inception/model_binv1.3')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/weights-nih/inception/model_binv1.3/assets
